# Create Containerlab topology from CYJS graph

Initialize parameters   

In [422]:
topology_name = "nr-1"

Read CYJS graph data into a dictonary and initialize networkx graph from it

In [423]:
import json
import networkx as nx

cyjs = {}
with open(topology_name + ".cyjs", 'r', encoding='utf-8') as f:
    cyjs = json.load(f)
G = nx.cytoscape_graph(cyjs)
print(G.graph)

{'name': 'DM-Akron'}


Parse graph G into lists of: nodes and links. Keep a list of interfaces per device in `device_interfaces_map`.

In [424]:
nodes, links = [], []
device_interfaces_map = {}
for n in G.nodes:
    if G.nodes[n]['type'] == 'device':
        dev = G.nodes[n]['device']
        nodes.append(dev)
        device_interfaces_map[dev['name']] = {}
    elif G.nodes[n]['type'] == 'interface':
        int_name = G.nodes[n]['interface']['name']
        dev_name, dev_node_id = None, None
        peer_name, peer_dev_name, peer_dev_node_id = None, None, None
        for a_adj in G.adj[n].items():
            if G.nodes[a_adj[0]]['type'] == 'device':
                dev_name = G.nodes[a_adj[0]]['device']['name']
                dev_node_id = G.nodes[a_adj[0]]['device']['node_id']
                device_interfaces_map[dev_name][int_name] = ""
            elif G.nodes[a_adj[0]]['type'] == 'interface' and G.nodes[n]['side'] == 'a':
                peer_name = G.nodes[a_adj[0]]['interface']['name']
                for b_adj in G.adj[a_adj[0]].items():
                    if G.nodes[b_adj[0]]['type'] == 'device':
                        peer_dev_name = G.nodes[b_adj[0]]['device']['name']
                        peer_dev_node_id = G.nodes[b_adj[0]]['device']['node_id']
        if G.nodes[n]['side'] == 'a':
            links.append({
                'a': {
                    'node': dev_name,
                    'node_id': dev_node_id,
                    'interface': int_name,
                },
                'b': {
                    'node': peer_dev_name,
                    'node_id': peer_dev_node_id,
                    'interface': peer_name,
                },
            })

print(nodes)
print(device_interfaces_map)
print(links)

[{'id': 1, 'type': 'device', 'name': 'dmi01-akron-rtr01', 'node_id': 0}, {'id': 14, 'type': 'device', 'name': 'dmi01-akron-sw01', 'node_id': 5}]
{'dmi01-akron-rtr01': {'GigabitEthernet0/1/0': '', 'GigabitEthernet0/1/1': ''}, 'dmi01-akron-sw01': {'GigabitEthernet1/0/1': '', 'GigabitEthernet1/0/2': ''}}
[{'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/0'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/1'}}, {'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/1'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/2'}}]


Create container-compatible interface names for each device. We assume interface with index `0` is reserved for management, and start with `1`

In [425]:
for node, map in device_interfaces_map.items():
    # sort keys (interface names) in the map
    map_keys = list(map.keys())
    map_keys.sort()
    sorted_map = {k: f"eth{map_keys.index(k)+1}" for k in map_keys}
    device_interfaces_map[node] = sorted_map

print(device_interfaces_map)

for l in links:
    l['a']['c_interface'] = device_interfaces_map[l['a']['node']][l['a']['interface']]
    l['b']['c_interface'] = device_interfaces_map[l['b']['node']][l['b']['interface']]

print(links)

{'dmi01-akron-rtr01': {'GigabitEthernet0/1/0': 'eth1', 'GigabitEthernet0/1/1': 'eth2'}, 'dmi01-akron-sw01': {'GigabitEthernet1/0/1': 'eth1', 'GigabitEthernet1/0/2': 'eth2'}}
[{'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/0', 'c_interface': 'eth1'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/1', 'c_interface': 'eth1'}}, {'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/1', 'c_interface': 'eth2'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/2', 'c_interface': 'eth2'}}]


Generate clab topology. Using this gist as inspiration https://gist.github.com/renatoalmeidaoliveira/fdb772a5a02f3cfc0b5fbe7e8b7586a2

In [426]:
topology = {
    'name': G.name,
    'nodes': [f"{n['name']}" for n in nodes],
    'links': [f"[\"{l['a']['node']}:{l['a']['c_interface']}\", \"{l['b']['node']}:{l['b']['c_interface']}\"]" for l in links],
}

print(json.dumps(topology, indent=4))

{
    "name": "DM-Akron",
    "nodes": [
        "dmi01-akron-rtr01",
        "dmi01-akron-sw01"
    ],
    "links": [
        "[\"dmi01-akron-rtr01:eth1\", \"dmi01-akron-sw01:eth1\"]",
        "[\"dmi01-akron-rtr01:eth2\", \"dmi01-akron-sw01:eth2\"]"
    ]
}


Load Jinja2 template for Containerlab to run the topology through

In [427]:
from jinja2 import Environment, FileSystemLoader
env = Environment(
            loader=FileSystemLoader(f"."),
            line_statement_prefix='#'
        )
templ = env.get_template(f"clab.j2")

Run the topology through jinja2 template to get the final result

In [428]:
topo = templ.render(topology)
print(topo)
with open(topology_name + ".clab.yml", "w") as f:
    f.write(topo)

name: DM-Akron
topology:
    nodes:
        graphite:
            kind: linux
            image: netreplica/graphite:nanog86
            env:
                CLAB_SSH_CONNECTION: ${SSH_CONNECTION}
            binds:
                - __clabDir__/topology-data.json:/htdocs/lab/default/topology-data.json:ro
                - __clabDir__/ansible-inventory.yml:/htdocs/lab/default/ansible-inventory.yml:ro
            ports:
                - 8080:80
            exec:
                - sh -c 'graphite_motd.sh 8080'
            labels:
                graph-hide: yes

        dmi01-akron-rtr01:
            kind: ceos
            image: ceos
            binds:
                - dmi01-akron-rtr01_interface_map.json:/mnt/flash/EosIntfMapping.json:ro
        dmi01-akron-sw01:
            kind: ceos
            image: ceos
            binds:
                - dmi01-akron-sw01_interface_map.json:/mnt/flash/EosIntfMapping.json:ro
    links:
        - endpoints: ["dmi01-akron-rtr01:eth1", "dmi01-akro

Interface mapping file for cEOS

In [429]:
ceos_interfaces_templ = env.get_template(f"interface_maps/ceos.j2")
for d, m in device_interfaces_map.items():
    print(d)
    print(m)
    ceos_interface_map = ceos_interfaces_templ.render({'map': m})
    print(ceos_interface_map)
    with open(d + "_interface_map.json", "w") as f:
        f.write(ceos_interface_map)

dmi01-akron-rtr01
{'GigabitEthernet0/1/0': 'eth1', 'GigabitEthernet0/1/1': 'eth2'}
{
  "ManagementIntf": {
    "eth0": "Management0"
  },
  "EthernetIntf": {
    "eth1": "GigabitEthernet0/1/0",
    "eth2": "GigabitEthernet0/1/1"
  }
}
dmi01-akron-sw01
{'GigabitEthernet1/0/1': 'eth1', 'GigabitEthernet1/0/2': 'eth2'}
{
  "ManagementIntf": {
    "eth0": "Management0"
  },
  "EthernetIntf": {
    "eth1": "GigabitEthernet1/0/1",
    "eth2": "GigabitEthernet1/0/2"
  }
}
